# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [4]:
amount_spent = orders.groupby(['CustomerID']).sum()
amount_spent.reset_index()

,CustomerID,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
0,12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
1,12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
2,12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
3,12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
4,12350,1365627,9231629,34187,34,51,272,197,65.30,334.40
...,...,...,...,...,...,...,...,...,...,...
4334,18280,1110495,5457120,20110,30,10,90,45,47.65,180.60
4335,18281,1560699,3895248,14077,42,49,70,54,39.36,80.82
4336,18282,4642134,6838540,24132,116,60,146,103,62.39,178.05
4337,18283,233950830,425704048,1520316,5503,2489,10346,1397,1220.93,2094.88


In [5]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

amount_spent['amount_spent'].quantile(0.75) #any customer between this and 0.95 is Preferred
amount_spent['amount_spent'].quantile(0.95) #any customer above is VIP

5840.181999999983

In [8]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

amount_spent['Customer'] = np.where(amount_spent['amount_spent'] >= amount_spent['amount_spent'].quantile(0.95), 'VIP Customer','')
amount_spent['Customer'] = np.where((amount_spent['amount_spent'] >= amount_spent['amount_spent'].quantile(0.75)) & (amount_spent['amount_spent'] < amount_spent['amount_spent'].quantile(0.95)), 'Preferred Customer', amount_spent['Customer'])

result = pd.concat([orders, amount_spent], axis=1, join='inner')
result['Customer']

12346          VIP Customer
12347    Preferred Customer
12348    Preferred Customer
12349    Preferred Customer
12350                      
                ...        
18280                      
18281                      
18282                      
18283    Preferred Customer
18287    Preferred Customer
Name: Customer, Length: 4339, dtype: object

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
# your code here
result['isVIP'] = np.where(amount_spent['amount_spent'] >= amount_spent['amount_spent'].quantile(0.95), 1,0)
result['isPRE'] = np.where((amount_spent['amount_spent'] >= amount_spent['amount_spent'].quantile(0.75)) & (amount_spent['amount_spent'] < amount_spent['amount_spent'].quantile(0.95)), 1, 0)

country = result.groupby('Country').sum()
country['isVIP'].nlargest(1)

Country
United Kingdom    191
Name: isVIP, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
# your code here
country = country['isVIP']+ country['isPRE']
country.nlargest(1)

Country
United Kingdom    971
dtype: int64